# Elastic Stackの構築手順：収容（AWS）

----

Elasticsearchのインストール先となる Amazon&nbsp;EC2インスタンスを確保します。  

## 設定
インスタンスの生成に必要な情報を設定します。

マシンイメージを設定します。

In [2]:
image_id = 'ami-eec1c380'
#CentOS 7 (x86_64) - with Updates HVM
#アジアパシフィック東京リージョン

インスタンスタイプを設定します。  
[Amazon EC2 インスタンス](https://aws.amazon.com/jp/ec2/instance-types/)のページから適切なタイプを選択してください。

例えば64GBのメモリを利用するのであれば **m4.4xlarge** が選択候補となります。  
利用したいサイズに応じてタイプを選んで下さい。

In [3]:
instance_type = 'm4.4xlarge'

どのロケーションにインスタンスを作成するかを設定します。

In [4]:
location = 'Asia Pacific (Tokyo)'

作成するインスタンス数を設定します。

In [5]:
instance_count = 3

データ蓄積ディスクとなる[EBS](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/AmazonEBS.html)の[ブロックデバイスマッピング](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/block-device-mapping-concepts.html)設定を行います。  
`VolumeSize`、`VolumeType`を設定してください。  
`DeleteOnTermination`にtrueを設定すると、インスタンス破棄時にEBSが削除されます。  


In [6]:
block_device_mappings = {
    "DeviceName": "/dev/sda1",
    "Ebs": {
      "VolumeSize": 1024,
      "VolumeType": "gp2",
      "DeleteOnTermination": "true",
    }
}

インスタンス群に付与する名前を設定します。  
この名前は、後の処理でタグのNameキーに設定されます。

In [7]:
instance_name='es-cluster'

[キーペア](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/ec2-key-pairs.html#having-ec2-create-your-key-pair)は、以下の公開鍵をインポートしておきます。

In [8]:
!cat ~/.ssh/ansible_id_rsa.pub

ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX


使用するキーペア名を設定します。

In [9]:
keypair_name = 'jupyter-aws'

セキュリティグループIDを設定してください。

In [16]:
security_group_id = 'sg-0adf9f6d'

以下、VPCと、必要ならばサブネットを設定します。    
デフォルトのVPCを利用するか否かで、次の
- デフォルトのVPCを利用する場合
- デフォルトのVPCを利用しない場合

のどちらかの節を実行してください。

<font color="red">このNotebookの最初の状態としては、[デフォルトのVPCを利用しない場合](#デフォルトのVPCを利用しない場合)の各コマンドはコメントアウトしています。</font>

### デフォルトのVPCを利用する場合  
セキュリティグループ名のみを設定してください。

In [10]:
security_group = 'juypter-group'

### デフォルトのVPCを利用しない場合
VPC内のサブネットIDと、セキュリティグループ**ID**（グループ名ではない）を設定してください。  
※それぞれのIDが既にわかっている場合はVPCの調査をスキップし、最後のID設定のセルのみ実施してください。

対象となるVPCを調べます。

In [11]:
#!aws ec2 describe-vpcs

{
    "Vpcs": [
        {
            "VpcId": "vpc-11c59c74", 
            "InstanceTenancy": "default", 
            "State": "available", 
            "DhcpOptionsId": "dopt-c337b8a6", 
            "CidrBlock": "172.31.0.0/16", 
            "IsDefault": true
        }
    ]
}


VPCのIDを設定してください。

In [12]:
#vpc_id = 'vpc-bd3b81d9'

設定したIDのVPCにあるサブネットを表示し、そのIDを調べます。

In [13]:
#!aws ec2 describe-subnets --filters "Name=vpc-id,Values=$vpc_id"

{
    "Subnets": [
        {
            "VpcId": "vpc-bd3b81d9", 
            "AvailableIpAddressCount": 4089, 
            "MapPublicIpOnLaunch": true, 
            "DefaultForAz": false, 
            "Ipv6CidrBlockAssociationSet": [], 
            "State": "available", 
            "AvailabilityZone": "ap-northeast-1c", 
            "SubnetId": "subnet-9646c9ce", 
            "CidrBlock": "172.31.16.0/20", 
            "AssignIpv6AddressOnCreation": false
        }, 
        {
            "VpcId": "vpc-bd3b81d9", 
            "AvailableIpAddressCount": 4091, 
            "MapPublicIpOnLaunch": false, 
            "DefaultForAz": false, 
            "Ipv6CidrBlockAssociationSet": [], 
            "State": "available", 
            "AvailabilityZone": "ap-northeast-1c", 
            "SubnetId": "subnet-8246c9da", 
            "CidrBlock": "172.31.0.0/20", 
            "AssignIpv6AddressOnCreation": false
        }
    ]
}


サブネットIDを設定してください。

In [15]:
#subnet_id = 'subnet-9646c9ce'

## EC2インスタンスの設定と生成 
サーバ構成に応じたEC2インスタンスの設定を定義し、インスタンスを生成します。

大まかな手順は次の通りです。  
1. AWSへのアクセスの確認
2. インスタンスの生成
3. セキュリティグループへの登録
4. タグの設定


### AWSへのアクセスの確認
AWSのCLIが実行可能か確認します。

具体的には、このNotebook環境に[AWS CLI](http://docs.aws.amazon.com/ja_jp/cli/latest/userguide/cli-chap-welcome.html)がインストールされており、[aws configure](http://docs.aws.amazon.com/ja_jp/streams/latest/dev/kinesis-tutorial-cli-installation.html#config-cli)にAWSの認証情報が登録されていることを確認します。  
設定がされていることの確認として、Availability Zoneを列挙してみます。

In [1]:
!aws ec2 describe-availability-zones

{
    "AvailabilityZones": [
        {
            "State": "available", 
            "RegionName": "ap-northeast-1", 
            "Messages": [], 
            "ZoneName": "ap-northeast-1a"
        }, 
        {
            "State": "available", 
            "RegionName": "ap-northeast-1", 
            "Messages": [], 
            "ZoneName": "ap-northeast-1c"
        }
    ]
}


JSON形式で出力されることを確認します。

### インスタンスの生成
設定した定義を用いてインスタンスを生成します。

In [17]:
import json
%run scripts/get-json-repr.py

# インスタンスを生成する。デフォルトのVPCを使うかどうか（上でsubnet_idを指定しているかどうか）でオプションを分ける
if('subnet_id' not in locals() or subnet_id is None):
    instances_stdout = !aws ec2 run-instances --image-id {image_id} --count {instance_count} --instance-type {instance_type} --key-name {keypair_name} --block-device-mappings {get_block_device_mappings_repr(block_device_mappings)} --security-groups {security_group}
else:
    instances_stdout = !aws ec2 run-instances --image-id {image_id} --count {instance_count} --instance-type {instance_type} --key-name {keypair_name} --block-device-mappings {get_block_device_mappings_repr(block_device_mappings)} --security-group-ids {security_group_id} --subnet-id {subnet_id}

instances = json.loads('\n'.join(instances_stdout))['Instances']
instances

[{u'AmiLaunchIndex': 2,
  u'Architecture': u'x86_64',
  u'BlockDeviceMappings': [],
  u'ClientToken': u'',
  u'EbsOptimized': False,
  u'Hypervisor': u'xen',
  u'ImageId': u'ami-eec1c380',
  u'InstanceId': u'i-062580d4adacade92',
  u'InstanceType': u'm4.4xlarge',
  u'KeyName': u'acro-estest_key',
  u'LaunchTime': u'2017-03-16T07:01:52.000Z',
  u'Monitoring': {u'State': u'disabled'},
  u'NetworkInterfaces': [{u'Attachment': {u'AttachTime': u'2017-03-16T07:01:52.000Z',
     u'AttachmentId': u'eni-attach-a19008cf',
     u'DeleteOnTermination': True,
     u'DeviceIndex': 0,
     u'Status': u'attaching'},
    u'Description': u'',
    u'Groups': [{u'GroupId': u'sg-0adf9f6d',
      u'GroupName': u'acro-sg-estest02'}],
    u'Ipv6Addresses': [],
    u'MacAddress': u'0a:3a:20:94:22:dd',
    u'NetworkInterfaceId': u'eni-5702c409',
    u'OwnerId': u'610531041244',
    u'PrivateDnsName': u'ip-172-31-30-87.ap-northeast-1.compute.internal',
    u'PrivateIpAddress': u'172.31.30.87',
    u'PrivateIpAdd

生成したインスタンスIDを確認します。

In [18]:
id_list = map(lambda i: i['InstanceId'].encode('utf-8'), instances)
id_list_text = ' '.join(id_list)
id_list

['i-062580d4adacade92', 'i-07261e8cde704e995', 'i-04f72c7db246797be']

インスタンスの状態が `running` になるまで待ちます。

In [19]:
from functools import reduce
import time

for loop in range(0, 10):
    # インスタンス情報を取得
    desc_instances_stdout = !aws ec2 describe-instances --instance-ids $id_list_text
    # JSON化し、インスタンスのリストとして取り出す
    desc_instances = json.loads('\n'.join(desc_instances_stdout))
    target_instances = reduce(lambda x, y: x + y, map(lambda r: r['Instances'], desc_instances['Reservations']))
    # IDと状態を出力
    status_text = '\n'.join(map(lambda i : '{0} : {1}'.format(i['InstanceId'], i['State']['Name']), target_instances))
    print(status_text)
    # runningになっていないものがまだあれば、スリープして最初に戻る。全てrunningなら終了。
    not_running_num = len(list(filter(lambda i : i['State']['Name'] != 'running', target_instances)))
    if(not_running_num == 0):
        break
    print('Not running : {0}'.format(not_running_num))
    time.sleep(5)
    print('------')

i-062580d4adacade92 : pending
i-07261e8cde704e995 : pending
i-04f72c7db246797be : pending
Not running : 3
------
i-062580d4adacade92 : pending
i-07261e8cde704e995 : pending
i-04f72c7db246797be : pending
Not running : 3
------
i-062580d4adacade92 : running
i-07261e8cde704e995 : running
i-04f72c7db246797be : running


確保したホストを確認します。

In [20]:
host_list = map(lambda i: i['PublicDnsName'].encode('utf-8'), target_instances)
host_list

['ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com',
 'ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com',
 'ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com']

pingが通ることを確認します。

In [21]:
import commands, sys
results = map(lambda host:commands.getoutput('ping -c 4 ' + host), host_list)
results

['PING ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com (54.238.216.212): 56 data bytes\n--- ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com ping statistics ---\n4 packets transmitted, 0 packets received, 100% packet loss',
 'PING ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com (54.249.8.131): 56 data bytes\n64 bytes from 54.249.8.131: icmp_seq=3 ttl=55 time=5.329 ms\n--- ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com ping statistics ---\n4 packets transmitted, 1 packets received, 75% packet loss\nround-trip min/avg/max/stddev = 5.329/5.329/5.329/0.000 ms',
 'PING ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com (54.249.98.66): 56 data bytes\n64 bytes from 54.249.98.66: icmp_seq=1 ttl=55 time=5.820 ms\n64 bytes from 54.249.98.66: icmp_seq=2 ttl=55 time=5.274 ms\n64 bytes from 54.249.98.66: icmp_seq=3 ttl=55 time=5.400 ms\n--- ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com ping statistics ---\n4 packets transmitted, 3 packets received, 25% packet 

全てのインスタンスに対しpingが通ればEC2インスタンス生成は完了です。

### セキュリティグループへの登録
各インスタンスのPrivate IPアドレスを、セキュリティグループの許可リストに追加します。  

インスタンスごとに、9200番ポート（外部通信用）と9300番ポート（クラスタ内部通信用）の両方を許可します。

In [22]:
private_ip_list = map(lambda i: i['PrivateIpAddress'].encode('utf-8'), instances)

for ip in private_ip_list:
    !aws ec2 authorize-security-group-ingress --group-id {security_group_id} --protocol tcp --port 9200 --cidr $ip/32
    !aws ec2 authorize-security-group-ingress --group-id {security_group_id} --protocol tcp --port 9300 --cidr $ip/32

### タグの設定
タグのNameキーに名称を設定します。

In [23]:
!aws ec2 create-tags --resources $id_list_text --tags Key=Name,Value=$instance_name

## Inventory作成用のインスタンスリストを作成
ホストのパブリックDNS一覧をリスト化します  

次のブックの[Inventory作成](01_03_Install.ipynb#Inventory作成)で使用します。  
出現した文字列をクリップボードにコピーしてください。

In [25]:
host_list = ['\'host{0}\':\'{1}\''.format(str(i+1), x) for i,x in enumerate(host_list)] 
print('host_list = {{{0}}}'.format(','.join(host_list)))

host_list = {'host1':'ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com','host2':'ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com','host3':'ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com'}
